# Hawkes Modeling\n

In [ ]:
import requests, numpy as np, matplotlib.pyplot as plt
BASE='http://mock_apis:8000'
tr = requests.get(f'{BASE}/api/binance/mock_trades/BTCUSDT?n=800').json()['trades']
T = np.array([t['time'] for t in tr], dtype=float); T = T - T.min()
try:
    from tick.hawkes import HawkesExpKern
    model = HawkesExpKern(decays=1.0)
    model.fit([T])
    model.plot()
except Exception as e:
    import matplotlib.pyplot as plt
    inter = np.diff(T)
    plt.hist(inter, bins=40); plt.title('Interarrival'); plt.show()
    print('tick not available:', e)
